In [1]:
import sqlite3
import pandas as pd

In [2]:
path = "../data/raw/h358-2015.sqlite3"

### 1 - SQL database exploration

In [3]:
class SQLiteConnection:
    """Context manager for SQLite3 connections."""
    
    def __init__(self, path):
        self.path = path
        self.conn = None
        self.cursor = None

    def __enter__(self):
        self.conn = sqlite3.connect(self.path)
        self.cursor = self.conn.cursor()
        return self.cursor

    def __exit__(self, exc_type, exc_value, traceback):
        if self.cursor:
            self.cursor.close()
        if self.conn:
            self.conn.close()


In [4]:
# fetch table names

with SQLiteConnection(path) as cursor:
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    table_names = cursor.fetchall()
    for table in table_names:
        print(table[0])

variable
data


In [5]:
# fetch "data" content

with SQLiteConnection(path) as cursor:
    cursor.execute('SELECT * FROM data')
    result = cursor.fetchall()
    data = []
    for row in result:
        data.append(row)

data[:5]

[(0, 1420676817200, 1.92, '08/01/2015 01:26:57'),
 (0, 1420964760498, 7.96, '11/01/2015 09:26:00'),
 (0, 1420964920406, 5.76, '11/01/2015 09:28:40'),
 (0, 1420965125280, 5.76, '11/01/2015 09:32:05'),
 (0, 1420965249865, 7.96, '11/01/2015 09:34:09')]

In [6]:
# fetch "data" column names

table_name = 'data'
with SQLiteConnection(path) as cursor:
    cursor.execute(f"PRAGMA table_info({table_name});")
    column_info = cursor.fetchall()
    column_names = [column[1] for column in column_info]

column_names

['variableidref', 'epochtimeinms', 'value', 'fulldate']

In [7]:
# fetch "variable" content

with SQLiteConnection(path) as cursor:
    cursor.execute('SELECT * FROM variable')
    result = cursor.fetchall()
    var_list = []
    for row in result:
        var_list.append(row)

var_list

[(0,
  'WIND_SPEED',
  'InpgViallet',
  'INPG_Viallet',
  'Weather station',
  'Emetteur'),
 (1,
  'ILLUMINANCE',
  'InpgViallet',
  'INPG_Viallet',
  'Weather station',
  'Emetteur'),
 (2,
  'DAY_NIGHT',
  'InpgViallet',
  'INPG_Viallet',
  'Weather station',
  'Emetteur'),
 (3, 'RAIN', 'InpgViallet', 'INPG_Viallet', 'Weather station', 'Emetteur'),
 (4,
  'TEMPERATURE',
  'InpgViallet',
  'INPG_Viallet',
  'Weather station',
  'Emetteur'),
 (5,
  'POWER',
  'InpgViallet',
  'INPG_Viallet',
  'H358 - Professor office',
  'PriseGigogneEltako5'),
 (6,
  'POWER',
  'InpgViallet',
  'INPG_Viallet',
  'H358 - Professor office',
  'PriseGigogneEltako4'),
 (7,
  'CONTACT',
  'InpgViallet',
  'INPG_Viallet',
  'H358 - Professor office',
  'Fenetre25'),
 (8,
  'CONTACT',
  'InpgViallet',
  'INPG_Viallet',
  'H358 - Professor office',
  'Porte18'),
 (9,
  'CONTACT',
  'InpgViallet',
  'INPG_Viallet',
  'H358 - Professor office',
  'Fenetre26'),
 (10,
  'POWER',
  'InpgViallet',
  'INPG_Viallet',

In [8]:
# fetch "variable" column names

table_name = 'variable'
with SQLiteConnection(path) as cursor:
    cursor.execute(f"PRAGMA table_info({table_name});")
    column_info = cursor.fetchall()
    column_names = [column[1] for column in column_info]

column_names

['id', 'name', 'site', 'building', 'zone', 'device']

In [9]:
# we will only consider sensors inside the "H358 - Professor office" room

trg_features = [v for v in var_list if v[4]=="H358 - Professor office"]
trg_features

[(5,
  'POWER',
  'InpgViallet',
  'INPG_Viallet',
  'H358 - Professor office',
  'PriseGigogneEltako5'),
 (6,
  'POWER',
  'InpgViallet',
  'INPG_Viallet',
  'H358 - Professor office',
  'PriseGigogneEltako4'),
 (7,
  'CONTACT',
  'InpgViallet',
  'INPG_Viallet',
  'H358 - Professor office',
  'Fenetre25'),
 (8,
  'CONTACT',
  'InpgViallet',
  'INPG_Viallet',
  'H358 - Professor office',
  'Porte18'),
 (9,
  'CONTACT',
  'InpgViallet',
  'INPG_Viallet',
  'H358 - Professor office',
  'Fenetre26'),
 (10,
  'POWER',
  'InpgViallet',
  'INPG_Viallet',
  'H358 - Professor office',
  'PriseGigogneEltako1'),
 (11, 'COV', 'InpgViallet', 'INPG_Viallet', 'H358 - Professor office', 'COV4'),
 (12,
  'POWER',
  'InpgViallet',
  'INPG_Viallet',
  'H358 - Professor office',
  'PriseGigogneEltako6'),
 (13,
  'CONTACT',
  'InpgViallet',
  'INPG_Viallet',
  'H358 - Professor office',
  'Fenetre27'),
 (14,
  'OCCUPANCY',
  'InpgViallet',
  'INPG_Viallet',
  'H358 - Professor office',
  'Presence2'),
 (

## 2 - Creating the new dataset
Since sensor observations are not captured at the same timestamps, we will resample the dataset and aggregate sensor values at a new sampling rate. 
A new column is created for each sensor.

In [10]:
data_div = []
for row in data:
    data_div.append([row[0], row[1], row[2], row[3]])
    
df = pd.DataFrame(data_div, columns=['variableidref', 'epochtimeinms', 'value', 'fulldate'])
df.drop('epochtimeinms', axis=1, inplace=True)
df['fulldate'] = pd.to_datetime(df['fulldate'], format='%d/%m/%Y %H:%M:%S')
df.sort_values(by='fulldate', inplace=True)

df.head()

,variableidref,value,fulldate
1856553,21,430.0,2015-01-01 00:00:28
1128977,19,30.0,2015-01-01 00:00:28
616337,11,16.0,2015-01-01 00:01:48
1856554,21,430.0,2015-01-01 00:05:51
1128978,19,30.0,2015-01-01 00:05:51


In [11]:
sample_rate = "30Min"

date_col = pd.DatetimeIndex(df['fulldate'].unique()).floor("30Min")
resampled_df = pd.DataFrame(index=date_col.unique())

for var_desc in trg_features:
    var_id = var_desc[0]
    var_name = var_desc[1]
    col_name = f"{var_name}_{var_id}"

    if var_name=="OCCUPANCY": # this is the target variable, we will not resample it
        values = df[df['variableidref'] == var_id].set_index('fulldate')['value'].resample(sample_rate).max()
    else:
        values = df[df['variableidref'] == var_id].set_index('fulldate')['value'].resample(sample_rate).mean()
    values.name = col_name
    resampled_df = resampled_df.join(values, how='left')

resampled_df.sort_index(inplace=True)
print(resampled_df.shape)
resampled_df.head()

(46975, 19)


,POWER_5,POWER_6,CONTACT_7,CONTACT_8,CONTACT_9,POWER_10,COV_11,POWER_12,CONTACT_13,OCCUPANCY_14,ILLUMINANCE_15,TEMPERATURE_16,POWER_17,TEMPERATURE_18,HUMIDITY_19,TEMPERATURE_20,GAS_CONCENTRATION_21,POWER_22,ILLUMINANCE_23
2015-01-01 00:00:00,0.0,16.0,0.0,NaN,0.0,13.0,16.166667,0.0,NaN,NaN,NaN,NaN,0.0,NaN,30.142857,18.766667,430.0,0.0,NaN
2015-01-01 00:30:00,0.0,16.0,NaN,0.0,NaN,NaN,16.750000,0.0,0.0,0.0,0.0,NaN,0.0,NaN,30.285714,18.800000,430.0,0.0,NaN
2015-01-01 01:00:00,0.0,16.0,NaN,NaN,NaN,13.0,16.375000,0.0,NaN,NaN,NaN,NaN,0.0,NaN,30.000000,18.800000,430.0,NaN,NaN
2015-01-01 01:30:00,0.0,16.0,0.0,NaN,0.0,13.0,16.142857,0.0,NaN,0.0,0.0,NaN,NaN,NaN,30.000000,18.727273,430.0,0.0,NaN
2015-01-01 02:00:00,0.0,16.0,NaN,0.0,NaN,13.0,16.428571,0.0,0.0,NaN,NaN,NaN,0.0,NaN,30.166667,18.706667,430.0,0.0,NaN


In [12]:
# NaN rate per column
for col in resampled_df.columns:
    print(f"{col}: {resampled_df[col].isna().sum() / len(resampled_df * 100):.2f}%")

# NaN rate per row
print(f"\nRows with NaN values: {resampled_df.isna().any(axis=1).sum() / len(resampled_df) * 100:.2f}%")

POWER_5: 0.20%
POWER_6: 0.13%
CONTACT_7: 0.13%
CONTACT_8: 0.16%
CONTACT_9: 0.14%
POWER_10: 0.13%
COV_11: 0.44%
POWER_12: 0.17%
CONTACT_13: 0.13%
OCCUPANCY_14: 0.14%
ILLUMINANCE_15: 0.13%
TEMPERATURE_16: 0.22%
POWER_17: 0.26%
TEMPERATURE_18: 0.20%
HUMIDITY_19: 0.44%
TEMPERATURE_20: 0.46%
GAS_CONCENTRATION_21: 0.44%
POWER_22: 0.13%
ILLUMINANCE_23: 0.20%

Rows with NaN values: 65.33%


In [13]:
# drop columns with NaN rate > threshold
threshold = 0.4
resampled_df.dropna(axis=1, thresh=len(resampled_df) * (1 - threshold), inplace=True)

In [14]:
# drop rows with NaN values

print(resampled_df.shape)
resampled_df.dropna(inplace=True)
print(resampled_df.shape)


(46975, 15)
(22498, 15)


In [15]:
# save new dataset to csv

resampled_df.to_csv("../data/processed/h358-2015.csv", index=True)